In [ ]:
crimes_df.createOrReplaceTempView("crimes")
descent_diction.createOrReplaceTempView("descent_diction")

query = """
WITH year_df AS (
    SELECT
        *,
        year(to_timestamp(`DATE OCC`, 'yyyy MMM dd hh:mm:ss a')) AS year
    FROM crimes
),
clean_df AS (
    SELECT
        *
    FROM year_df
    WHERE year IS NOT NULL
      AND `Vict Descent` IS NOT NULL
),
per_race AS (
    SELECT
        year,
        `Vict Descent`,
        COUNT(*) AS cnt
    FROM clean_df
    GROUP BY year, `Vict Descent`
),
ranked AS (
    SELECT
        year,
        `Vict Descent`,
        cnt,
        SUM(cnt) OVER (PARTITION BY year) AS year_total,
        ROUND(cnt / SUM(cnt) OVER (PARTITION BY year) * 100, 1) AS percent,
        DENSE_RANK() OVER (PARTITION BY year ORDER BY cnt DESC) AS rank
    FROM per_race
),
top3 AS (
    SELECT *
    FROM ranked
    WHERE rank <= 3
),
final AS (
    SELECT
        t.year,
        d.`Vict Descent Full` AS Vict_Descent,
        t.cnt,
        t.percent
    FROM top3 t
    LEFT JOIN descent_diction d
      ON t.`Vict Descent` = d.`Vict Descent`
)
SELECT
    *
FROM final
ORDER BY year DESC, cnt DESC
"""

result_sql = spark.sql(query)
result_sql.show(5, False)